In [46]:
from torch import cuda
import torch.nn as nn
import transformers
from transformers import DistilBertTokenizer, DistilBertModel
import warnings
import torch
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [14]:
class BERT(nn.Module):
    def __init__(self, bert):
        
        super(BERT, self).__init__()
        
        self.bert = bert
        self.dropout = nn.Dropout(0.1)
        self.relu =  nn.ReLU()
        self.fc1 = nn.Linear(768, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, **kwargs):

        cls_hs = self.bert(**kwargs)
        hidden_state = cls_hs[0]
        pooler = hidden_state[:, 0]
        
        x = self.fc1(pooler)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)

        return self.fc1(pooler)

In [15]:
model = BERT(bert)

try:
    model_save_name = 'saved_weights_BERT_description_classifier.pt'
    path = "../../../models/saved_weights/" + model_save_name
    model.load_state_dict(torch.load(path, 
                                        map_location=torch.device('cpu')))
    print('Local Success')
except:
    raise ValueError

model = model.to(device)

Local Success


In [39]:
def embeddings(span, model, pred_values=False, truncation=True, threshold=False):
    with torch.no_grad():
        inputs = tokenizer(span, return_tensors="pt", truncation=truncation)
        return model(**inputs)

def similarity(GroundTruth, Prediction):
    GroundTruth = embeddings(GroundTruth, model=model)
    Prediction = embeddings(Prediction, model=model)
    return cosine_similarity(GroundTruth, Prediction)[0][0]

In [57]:
root = "../../../data/interim/"

df_andrei = pd.read_csv(root + "DF_Andrei.csv", header=[0, 1], index_col=0) 
df_pierre = pd.read_csv(root + "DF_Pierre.csv", header=[0, 1], index_col=0) 
df_daniel = pd.read_csv(root + "DF_Daniel.csv", header=[0, 1], index_col=0) 

### Load Sentences

In [31]:
root = "../../../data/processed/"
sentences_all = {}

f = open(F"{root}Sentences_Pierre.pkl", 'rb')
sentences_Pierre = pickle.load(f)
sentences_all |= sentences_Pierre 

f = open(F"{root}Sentences_Andrei.pkl", 'rb')
sentences_Andrei = pickle.load(f)
sentences_all |= sentences_Andrei 

f = open(F"{root}Sentences_Kissling.pkl", 'rb')
sentences_Kissling = pickle.load(f)
sentences_all |= sentences_Kissling 

# Drop duplicates
for species, sentences in sentences_all.items():
    sentences_all[species] = list(set(sentences))

### Match Data

In [61]:
# k= 5
# google_form_lst = []

# for idx, (species, sentences) in enumerate(tqdm(sentences_all.items(), leave=True, desc="Species")):

#     # if idx > 1:
#     #     continue
#     if species != 'Dypsis thiryana':
#         continue
    
#     df_select = [df_andrei, df_pierre, df_daniel]
#     if species in df_andrei.index:
#         df_select = df_select[0]
#     elif species in df_pierre.index:
#         df_select = df_select[1]
#     elif species in df_daniel.index:
#         df_select = df_select[2]
#     else:
#         raise ValueError

#     for gt_main_trait in df_select.columns.get_level_values(0).unique():
#         if gt_main_trait == 'Measurement':
#             df_subset = df_select[df_select.index == species][gt_main_trait]
#             # Not really efficient, use PD?
    
#             subtraits = list(df_subset.columns)
#             values = df_subset.values[0]
#             for sub_trait, value in zip(subtraits, values):

#                 df_sent  = F"{sub_trait}: {value}"

#                 top_list = []

#                 for sentence in tqdm(sentences, leave=False, desc="Sentences"):
#                     gt_sim = similarity(df_sent, sentence)
#                     if not gt_sim:
#                         sentence = np.NaN
#                     top_list.append((gt_sim, sentence))

#                 top_list.sort(reverse=True)
#                 # top_k_list = [sentence for (_, sentence) in top_list[0:k]]
#                 gt_sim_sum = 0
#                 top_k_list = []
#                 for (gt_sim, sentence) in top_list[0:k]:
#                     gt_sim_sum += gt_sim
#                     top_k_list.append(sentence)

#                 # Non Nan for normalization
#                 None_NaNs = k - top_k_list.count(np.NaN) + 2e-26 # Float division               

#                 google_form_lst.append((species, gt_main_trait, sub_trait, [df_sent], *top_k_list, df_sent.capitalize(), gt_sim_sum/None_NaNs))
#         else:
#             df_subset = df_select[df_select.index == species][gt_main_trait]
#             present_traits = df_subset.loc[:, df_subset.any()].columns.values
            
#             # df_sent  = ' '.join(gt_main_trait + ' ' + present_traits)
            
#             # NEW VERSION SINGLE TRAIT
#             size = present_traits.shape
#             if not size[0]:
#                 continue
#             df_sent = F"{gt_main_trait} {present_traits[0]}"
#             # print(df_sent)

#             top_list = []

#             for sentence in tqdm(sentences, leave=False, desc="Sentences"):
#                 gt_sim = similarity(df_sent, sentence)

#                 if not gt_sim:
#                     sentence = np.NaN

#                 # print(gt_sim, sentence, df_sent)
#                 top_list.append((gt_sim, sentence))

#             top_list.sort(reverse=True)

#             # print(top_list)

#             # top_k_list = [sentence for (_, sentence) in top_list[0:k]]
#             gt_sim_sum = 0
#             top_k_list = []
#             for (gt_sim, sentence) in top_list[0:k]:
#                 gt_sim_sum += gt_sim
#                 top_k_list.append(sentence)

#             # Non Nan for normalization
#             None_NaNs = k - top_k_list.count(np.NaN) + 2e-26 # Float division       

#             # GoogleSent = gt_main_trait + ': ' + ', '.join(list(present_traits))
#             GoogleSent = F"{gt_main_trait}: {present_traits[0]}"
#             google_form_lst.append((species, gt_main_trait, gt_main_trait, list(present_traits), *top_k_list, GoogleSent.capitalize(), gt_sim_sum/None_NaNs))

Species: 100%|██████████| 647/647 [00:12<00:00, 50.31it/s]


In [62]:
# Google forms
df_google = pd.DataFrame(google_form_lst, columns=['Species', 'Main Trait', 'GT Main Trait', 'GT Sub Traits', '1', '2', '3', '4', '5', 'GoogleSentence', 'Sim Sum'])
df_google

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Sim Sum
0,Dypsis thiryana,Fruit Colour Description,Fruit Colour Description,[red],Rank: FLOWERING.,Rank: SEEDLING.,"The from is extremely thin, almost less than a...",.The larger one is soon to have a pair of infl...,Leaves. pinnate with two or more leaflets.,Fruit colour description: red,0.826880
1,Dypsis thiryana,Fruit Colour,Fruit Colour,[red],Rank: FLOWERING.,Leaves. pinnate with two or more leaflets.,"Clustering palm, in tufts of 2-4.","The from is extremely thin, almost less than a...",Clustering palm in tufts of 2-4.,Fruit colour: red,0.838760
2,Dypsis thiryana,Crown,Crown,[understorey],Rank: FLOWERING.,Groundcovers.,Clear.,Present.,Sans.,Crown: understorey,0.826209
3,Dypsis thiryana,Fruit Size,Fruit Size,[small],"Clustering palm, in tufts of 2-4.",Leaves. pinnate with two or more leaflets.,Clustering palm in tufts of 2-4.,Rank: FLOWERING.,"Pinnate leaves, leaflets unequally jagged and ...",Fruit size: small,0.855436
4,Dypsis thiryana,Fruit Shape,Fruit Shape,[globose],"Clustering palm, in tufts of 2-4.",Clustering palm in tufts of 2-4.,Rank: FLOWERING.,Leaves. pinnate with two or more leaflets.,"Pinnate leaves, leaflets unequally jagged and ...",Fruit shape: globose,0.867743
5,Dypsis thiryana,Conspicuousness,Conspicuousness,[conspicuous],Groundcovers.,Rank: FLOWERING.,Sans.,Present.,Rank: SEEDLING.,Conspicuousness: conspicuous,0.778821
6,Dypsis thiryana,Measurement,Maximum Stem Height in Meters,[Maximum Stem Height in Meters: 1.0],This is an extremely thin palm with extremely ...,Rank: SEEDLING.,Rank: FLOWERING.,"The from is extremely thin, almost less than a...","Clustering palm, in tufts of 2-4.",Maximum stem height in meters: 1.0,0.829054
7,Dypsis thiryana,Measurement,Maximum Stem Diameter in Centimeters,[Maximum Stem Diameter in Centimeters: 0.6],Leaves. pinnate with two or more leaflets.,"Pinnate leaves, leaflets unequally jagged and ...",This is an extremely thin palm with extremely ...,"Clustering palm, in tufts of 2-4.",Clustering palm in tufts of 2-4.,Maximum stem diameter in centimeters: 0.6,0.853949
8,Dypsis thiryana,Measurement,Maximum Leaf Number,[Maximum Leaf Number: 10.0],Leaves. pinnate with two or more leaflets.,Rank: FLOWERING.,"The from is extremely thin, almost less than a...","Pinnate leaves, leaflets unequally jagged and ...",This is an extremely thin palm with extremely ...,Maximum leaf number: 10.0,0.882855
9,Dypsis thiryana,Measurement,Maximum Leaf Blade Length in Meters,[Maximum Leaf Blade Length in Meters: 0.48],Leaves. pinnate with two or more leaflets.,"Pinnate leaves, leaflets unequally jagged and ...",This is an extremely thin palm with extremely ...,"The from is extremely thin, almost less than a...","Clustering palm, in tufts of 2-4.",Maximum leaf blade length in meters: 0.48,0.874254


### Google DF

In [77]:
root = "../../../data/processed/"
df_google = pd.read_csv(root + "top_sents_all_AllSentencesAgainstTrait_BERT_AllData.csv", index_col=0) 

root = "../../../data/interim/"
df_google.to_csv(root + "DF_GoogleForms_Bert_AllData.csv")

In [69]:
df_google\
    .dropna()\
    .sort_values(by=['Sim Sum'], ascending=False)

,Species,Main Trait,GT Main Trait,GT Sub Traits,1,2,3,4,5,GoogleSentence,Sim Sum
1815,Clausena anisata,inflorescences,inflorescences,['Panicle of flowers or inflorescences'],"Flowers Inflorescence axillary racemes, flower...","Flowers : Inflorescence axillary racemes , flo...","Flowers in axillary, cymose panicles, yellow o...",Flowers: Bunches of small creamy-yellow âsta...,"Inflorescence of elongate puberulous panicles,...",Inflorescences: panicle of flowers or inflores...,0.966772
7952,Bursera simaruba,Leaf composition,Leaf composition,['5-9 pinnate'],"Leaf type: odd-pinnately compound, made up of ...",Leaf shape: ovate to oblong.,"Leaf blade length: 4 to 8 inches, leaflets are...","Leaves: Alternate , pinnately compound, 6-8"" Ã...",The leaves are compound ranging from 3 to 9 le...,Leaf composition: 5-9 pinnate,0.964268
8528,Randia aculeata,Leaf shape,Leaf shape,"['spathulate, obovate']",Leaf shape: orbiculate.,Leaf venation: pinnate.,Leaves opposite on main branches but congested...,Leaf type: simple.,Leaf margin: entire.,"Leaf shape: spathulate, obovate",0.963610
5958,Prosopis juliflora,Leaf composition,Leaf composition,"['bi-pinnate, 20-40 leaflets per jug']","Leaves, are twice-compound with mostly two, so...","Leaves bipinnate, glabrous or pubescent, 1-3 p...","Â Leaves, petiolate 1-4 cm, bipinnate with gl...","Leaves medium to large, 8â16 cm long.",Leaves arising from very condensed short-shoot...,"Leaf composition: bi-pinnate, 20-40 leaflets p...",0.962056
5959,Prosopis juliflora,Leaf shape,Leaf shape,"['linear, obovate']","Leaflets linear -oblong, apex sub-obtuse, mucr...","Leaves, are twice-compound with mostly two, so...","Leaves alternate, bipinnate, with 1-3 pairs of...","Leaflets oblong , apex obtuse , pubescent, 10â...",Leaflets oblong and sub-pubescent.,"Leaf shape: linear, obovate",0.961554
...,...,...,...,...,...,...,...,...,...,...,...
2432,Croton psedopulchellus,sexuality,sexuality,['Terminal sexuality'],Seed germination takes 3-4 weeks.,"A pale white/yellow-flowered, shrubby, perenni...",Male flowers:pedicels 2â3 mm.,"A much-branched shrub, or small tree up to 5 m...",The flowers are in compact heads at the ends o...,Sexuality: terminal sexuality,0.647814
1383,Cajanus kerstingii,sexuality,sexuality,['Axillary sexuality'],The leaves are faintly striate and a glandular...,"Perennial, Not climbing, Shrub.",Morphology Branchlets and undersurface of leav...,A shrub 3â7 ft. high with stems less angular...,Branchlets and undersurface of leaves shortly ...,Sexuality: axillary sexuality,0.640310
218,Acacia kirkii,sexuality,sexuality,['Axillary sexuality'],Sans.,"It has a spreading, flat-topped crown, growing...",Vachellia kirkii is a multi-trunked shrub or t...,"Seeds blackish-olive, 5-7 x 4Â·5-5Â·5 mm.",Leaves: rhachis 3-8 cm.,Sexuality: axillary sexuality,0.638355
5792,Pericopsis laxiflora,sexuality,sexuality,"['Axillary sexuality', 'Terminal sexuality']",Definition: whole plant arborescent.,"A tree or shrub, common growth reaches between...","Trunk usually twisted, rarely straight and wit...","Leaves, alternate arrangement, 4 - 6 pairs, up...","Leaves, alternate arrangement, 4 - 6 pairs, up...",Sexuality: axillary sexuality,0.627646


In [75]:
df_google_thresh = df_google[df_google["Sim Sum"] > 0.9]

for i in range(1, 11, 2):
    df_google_subset = df_google_thresh\
                                     .sample(n=20, axis=0, random_state=333+i)

    df_google_subset.to_csv(f'{root}top_sents_all_AllSentencesAgainstTrait_Random20Subset_{i+1}.csv', sep='\t')

df_google_subset = df_google[df_google["Sim Sum"] > 0.9].sample(n=20, axis=0, random_state=123)
df_google_subset.to_csv(f'{root}top_sents_all_AllSentencesAgainstTrait_Random20Subset_demo_BERT.csv', sep='\t')